## Rate-coded version of the dual pathway model

This script creates a sensorimotor landscapes by superimposing several Gaussians, and tests the dual pathway model on it.

The model consists of two parallel pathways:-
- one: performs RL and is reset every day (exploration)
- two: maintains a record of the BG exploration (exploitation)

The reset at night depends on the potentiation observed during the day.

In [19]:
# Importing relevant packages

import gc
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from tqdm import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable


##### Auxilliary functions

Functions to generate the sensorimotor landscape on which the model is tested

In [2]:
def hill(sigma=0.1, center=None):
        """ Randomly assigns the mean and std deviation for the hills in the reward contour. """

        if center is None:
            # r = np.sqrt(np.random.uniform(0.0, 1.0))
            # a = np.random.uniform(0, 2*np.pi)
            # center = r*np.cos(a), r*np.sin(a)
            center = np.random.uniform(-1, 1, 2)

        return center, sigma

In [3]:
def gaussian(n=128, center=(0,0), sigma=0.1):
        """ Creates a 2D gaussian distribution, given the center coordinates and std deviation. """

        X, Y = np.meshgrid(np.linspace(-1, +1, n), np.linspace(-1, +1, n))
        x0, y0 = center
        D = np.sqrt((X-x0)**2/(2*sigma**2) + (Y-y0)**2/(2*sigma**2))

        return 1/(2*np.pi*sigma**2)*np.exp(-D)

In [1]:
def generate_gradient_artificial(n=256, n_distractors=20):
        """ Creates the overall reward contour by combining several gaussians. """
        
        # No. of local optima
        n_hills = n_distractors

        # Target i.e. global optima
        # target_theta = np.random.uniform(0,2*np.pi)
        # target_r = np.random.uniform(0,1)
        # targetpos = target_r*np.cos(target_theta), target_r*np.sin(target_theta)
        targetpos = np.random.uniform(-1, 1, 2)
        hills = [hill(0.3, targetpos)] # chosen sigma=0.3

        # n_hills distractors (0.4 < sigma < 0.7) i.e. local optima
        for i in range(n_hills):
            hills.append(hill(np.random.uniform(0.4, 0.7)))

        # Build gradient landscape
        Z = np.zeros((n,n))
        for (center, sigma) in hills:
            Z = np.maximum(Z, gaussian(n, center, sigma))
            # Z = Z + gaussian(n, center, sigma)    # For a smoother reward profile    
        
        return  Z / Z.max(), targetpos

In [20]:
def generate_gradient(n=256, contour_type='Artificial'):
        """ Redirects to the function that generates the desired contour type """
        
        if contour_type == 'Syrinx': return generate_gradient_syrinx(n)
        elif contour_type == 'Artificial': return generate_gradient_artificial(n)
        else: print("Contour type not specified (Syrinx/Artificial).")

In [4]:
def read_gradient(Z, p):
        """ Returns height of reward profile at given position. """

        x = min(max(p[0], -1), 1)
        y = min(max(p[1], -1), 1)
        col = int(((x + 1)/2) * (Z.shape[1]-1))
        row = int(((y + 1)/2) * (Z.shape[0]-1))

        return Z[row, col]

Activation functions.

Steep sigmoidals help in making neurons function as essentially binary.

In [5]:
def sigmoid(x, m=5, a=0.5):
    """ Returns an output between 0 and 1. """
    return 1 / (1 + np.exp(-1*(x-a)*m))

In [6]:
def sigmoid_neg(x, m=5, a=0.5):
    """ Returns an output between -1 and 1. """
    return (2 / (1 + np.exp(-1*(x-a)*m))) - 1

Learning rate of motor pathway.

Vestigial code.
Used previously to test changing learning rates in the cortical pathway over development.

In [7]:
def pPos(t):
    """ Testing changing learning rate of HL weights """ 

    return 0.00002

##### Main function to simulate learning

In [8]:
# FIX

In [9]:
def build_and_simulate(arg_rseed, arg_annealing=True, arg_plot=True):
    """ Main function to build and simulate the model """

    gc.collect()

    # Parameters
    # ---------- #

    # Random seed parameters
    rSeed = int(arg_rseed)
    np.random.seed(rSeed)

    # Experiment parameterss
    annealing = arg_annealing
    plotting = arg_plot

    # Output parameters
    resname = str(rSeed)
    if annealing == True: resname = resname + '_annealing'

    print(resname)

    # Layer parameters
    HVC_total_size = 100              
    BG_size = 50
    # RA_inh_size = 2
    RA_size = 100
    MC_size = 2
    n_BG_clusters = 10
    n_RA_clusters = MC_size

    # Weight parameters
    soft_bound = False                          
    Wmin_f, Wmax_f = 0, 1
    Wmin_RL, Wmax_RL = -1, 1
    Wmin_HL, Wmax_HL = -1, 1
    Wepsilon = 0.1

    # Input encoding
    n_syllables = 1                                            # setting the no. of syllables to simulate; keep in mind the maximum syllables possible without overlap in encoding
    n_bits = HVC_total_size//10                                # no. of bits active in each syllable encoding; corresponds with 10% of HVC being active for one timepoint in song
    HVC_size = n_bits * n_syllables                            # effective size of HVC


    # Simulation parameters
    n_daily_motifs = 1000
    n_days = 60
    n_trial_per_syll = n_daily_motifs * n_days 
    n_intact_trials = n_trial_per_syll * n_syllables            # Both pathways are active in intact trials
    n_lesioned_days = 1                                         # to check Hebbian learning result without BG input
    n_total_trials = n_intact_trials + n_lesioned_days * n_daily_motifs * n_syllables

    # Layer interactions
    RA_inter_neurons = False                                             # To include inhibitory inter-neurons in RA
    Hebbian_learning = HEBBIAN_LEARNING = True
    BG_influence = True
    Hebbian_rule = 1                                                     # Type of learning rule: 1 -> Hebbian, 2 -> iBCM, 3 -> Oja
    eta = ETA = 0.1

    # Neuron parameters
    BG_noise_mean = 0.00                                                 # Activation function parameters
    BG_noise_std = 0.05
    RA_noise_mean = 0.00
    RA_noise_std = 0.01

    BG_sig_slope = 10                                                    # BG sigmoidal should be as less steep as possible
    BG_sig_mid = 0
    RA_sig_slope = 30                                                    # RA sigmoidal should be as steep as possible
    RA_sig_mid = 0

    # Reward computation parameters: performance is computed with a window over recent trials
    reward_window = 25                      
    reward_sigma = 0.2


    # Model build
    # ----------- #
    HVC = np.zeros(HVC_size)
    RA = np.zeros(RA_size)
    MC = np.zeros(MC_size)
    BG = np.zeros(BG_size)

    # Randomised initialisation
    W_HVC_RA = np.zeros((HVC_size, RA_size), float) #+ .5 # Wepsilon                                        # HVC-RA connections should start from non-existent;
    W_HVC_BG = np.random.uniform(Wmin_RL + Wepsilon, Wmax_RL - Wepsilon, (HVC_size, BG_size))               # HVC-BG randomly initialised
    W_BG_RA = np.random.uniform(Wmin_f + Wepsilon, Wmax_f - Wepsilon, (BG_size, RA_size))                   # BG-RA generated randomly and remains fixed over the simulation
    W_RA_MC = np.random.uniform(Wmin_f + Wepsilon, Wmax_f - Wepsilon, (RA_size, MC_size))                   # RA-MC generated randomly and remains fixed over the simulation

    # Segregated pathways between RA and MC (topological connections)
    RA_cluster_size = RA_size//n_RA_clusters
    BG_cluster_size = BG_size//n_BG_clusters
    RA_channel_size = RA_size//n_BG_clusters

    for i in range(n_RA_clusters):
        segPath = np.diag(np.ones(n_RA_clusters, int))[i]
        W_RA_MC[i*RA_cluster_size : (i+1)*RA_cluster_size] *= segPath

    # Segregated pathways between BG and RA
    for i in range(n_BG_clusters):
        segPath = np.diag(np.ones(n_BG_clusters, int))[i]
        W_BG_RA[i*BG_cluster_size : (i+1)*BG_cluster_size] *= [j for j in segPath for r in range(RA_channel_size)]

    # Sparsity increases as number of connections/neurons increases
    # idx1 = np.random.choice(np.arange(W_HVC_BG.size), size=int(np.sqrt(W_HVC_BG.size)), replace=False)
    # W_HVC_BG[idx1//BG_size, idx1%BG_size] = 0
    idx = np.random.choice(np.arange(W_BG_RA.size), size=int(np.sqrt(W_BG_RA.size)), replace=False)
    W_BG_RA[idx//RA_size, idx%RA_size] = 0
    idx = np.random.choice(np.arange(W_RA_MC.size), size=int(np.sqrt(W_RA_MC.size)), replace=False)
    W_RA_MC[idx//MC_size, idx%MC_size] = 0


    ## Syllable encoding (input)
    syllable_encoding = {}
    syllables = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]              # General repertoire
    syllables = syllables[:n_syllables]                                         # ensure n_samples >= len(syllables)

    # Eg. 3 bit encoding for 2 syllables in a 10 neuron HVC:- "A" 1110000000, "B" 0001110000
    for i in range(n_syllables):
        inputs = np.zeros(HVC_size)
        inputs[i * n_bits:(i + 1) * n_bits] = 1
        syllable_encoding[syllables[i]] = inputs

    # Different reward landscape for each syllable
    syllable_targets = {}
    syllable_contours = {}
    for i in range(n_syllables):
        contour, target = generate_gradient()
        syllable_contours[syllables[i]] = contour
        syllable_targets[syllables[i]] = target



    # ---------- #

    # Tracking
    R = np.zeros((n_syllables, n_total_trials//n_syllables))                                # keeps track of reward
    W_RL = np.zeros((n_total_trials, n_bits*BG_size))                                       # tracks HVC-BG weights
    W_HL = np.zeros((n_total_trials, W_HVC_RA.size))                                        # tracks HVC-RA weights

    S_MO = np.zeros((n_total_trials, MC_size))                                              # tracks the 2D output
    S_T = np.zeros((n_total_trials, MC_size))                                               # tracks target
    
    S_RA = np.zeros((n_total_trials, RA_size))                                              # tracks RA activity
    S_BG = np.zeros((n_total_trials, BG_size))                                              # tracks BG activity

    S_dW1_sum = np.zeros((n_syllables, n_total_trials//n_syllables))                        # tracks sum of change in BG weights over a given day (proxy for measure of potentiation over the day)
    S_p = np.zeros((n_total_trials))                                                        # function of S_dW1_sum

    S_RA_HVC = np.zeros((n_total_trials, RA_size))                                          # tracks input to each RA neuron from the HVC
    S_RA_BG = np.zeros((n_total_trials, RA_size))                                           # tracks input to each RA neuron from the BG

    sum_W_HVC_BG = np.sum((W_HVC_BG!=0), axis=0)                                            # Used for normalisation
    sum_W_BG_RA = np.sum(np.abs(W_BG_RA), axis=0)                                           # Used for normalisation
    sum_W_RA_MC = np.sum(W_RA_MC, axis=0)                                                   # Used for normalisation


    RA_HVC = np.zeros((RA_size))                                                            # current input to each RA neuron from the HVC
    RA_BG = np.zeros((RA_size))                                                             # current input to each RA neuron from the BG
    dW_RL = np.zeros(W_HVC_BG.shape)                                                        # current change to RL weights
    dW_HL = np.zeros(W_HVC_RA.shape)                                                        # current change to HL weights
    night_noise = np.zeros(W_HVC_BG.shape)                                                  # nightly noise induced within RL weights
    dW_night = np.zeros(W_HVC_BG.shape)                                                     # nightly displacement of RL weights


    #### Main simulation of learning
    nt = 0
    inh_lesion = 1
    
    # Simulate over a some training days + one day with BG lesioned off
    for n_day in np.arange(n_days + n_lesioned_days):
        day_dW1_sum = np.zeros((n_syllables))                                               # Tracks BG weights potentiation over the day
        day_R_sum = np.zeros((n_syllables))                                                 # Tracks reward obtained over the day
        
        # Turn BG off if training is over
        if BG_influence==1 and n_day >= n_days:
            BG_influence = 0
            
        # Simulate a given number of motifs each day
        for n_motif in np.arange(n_daily_motifs):
             
            # (1 motif = 1 iteration over all syllables)
            for n_syll in np.arange(n_syllables):
                # Tracking iteration number wrt motifs and syllables
                n_iter = int(n_day * n_daily_motifs + n_motif)
                nt = n_iter * n_syllables + n_syll

                # Syllable input and target encoding
                syll = syllables[n_syll]                                                    
                target = syllable_targets[syll]
                contour = syllable_contours[syll]
                HVC[...] = syllable_encoding[syll]

                
                # Compute BG
                BG[...] = np.dot(HVC, W_HVC_BG) / sum_W_HVC_BG + np.random.normal(BG_noise_mean, BG_noise_std, BG_size)
                BG[...] = sigmoid_neg(BG, BG_sig_slope, BG_sig_mid)
            

                # Compute RA activity
                RA_HVC[...] = np.dot(HVC, W_HVC_RA) / n_bits * Hebbian_learning
                RA_BG[...] = np.dot(BG, W_BG_RA) / sum_W_BG_RA * BG_influence * 2
        
                RA =  ((RA_HVC) * Hebbian_learning) + (RA_BG * BG_influence)
                
                RA[...] = sigmoid_neg(RA, RA_sig_slope, RA_sig_mid)

                RA_noise = np.random.normal(RA_noise_mean, RA_noise_std, RA_size)                                       # Currently this has been set to extremely low - so for now, it is ineffective
                RA = RA + RA_noise

                # Compute MC activity
                MC[...] = (np.dot(RA, W_RA_MC)/sum_W_RA_MC)

                # Print 2D output at first trial (initial point)
                if nt == 0: print(MC)                                       
                

                # Compute error and reward
                reward = read_gradient(contour, MC)


                # Computing mean recent reward
                R_ = 0
                if n_iter > reward_window: R_ = R[n_syll, n_iter - reward_window:n_iter].mean()
                elif n_iter > 0: R_ = R[n_syll, :n_iter].mean()

                # Weight update
                dW_RL[...] = eta * (reward - R_) * HVC.reshape(HVC_size, 1) * (BG) * BG_influence


                # Tau for lBCM (Law and Cooper, 1994)
                theta_M = 1
                
                
                if Hebbian_rule == 1:
                    dW_HL[...] = pPos(n_iter) * HVC.reshape(HVC_size, 1) * (RA) * Hebbian_learning

                elif Hebbian_rule == 2:
                    if nt > 500:    theta_M = np.power(np.mean(S_RA[nt-500:nt], axis=0)-.5, 2)
                    elif nt > 0:    theta_M = np.power(np.mean(S_RA[:nt], axis=0)-.5, 2)
                    dW_HL[...] = pPos(n_iter) * HVC.reshape(HVC_size, 1) * (RA) / (theta_M+Wepsilon) * (RA-theta_M) * Hebbian_learning                                                                  # iBCM

                elif Hebbian_rule == 3:
                    dW_HL[...] = pPos(n_iter) * ((HVC.reshape(HVC_size, 1) * RA) - (HVC.reshape(HVC_size, 1) * RA * RA * W_HVC_RA)) * Hebbian_learning                                                 # Oja learning rule                                                                  RA_size) * Hebbian_learning

                # Track potentiation received
                day_dW1_sum[n_syll] += np.mean((np.abs(dW_RL))[n_bits*(n_syll):n_bits*(n_syll+1)])

                # Bounding weights (asymptotically/abruptly)
                if soft_bound == 1:
                    # W_HVC_BG += dW_RL * (Wmax_RL - W_HVC_BG) * (W_HVC_BG - Wmin_RL)
                    W_HVC_RA[...] += dW_HL * (Wmax_HL - W_HVC_RA) * (W_HVC_RA - Wmin_HL)
                else:
                    W_HVC_RA[...] = np.minimum(Wmax_HL, np.maximum(Wmin_HL, W_HVC_RA + dW_HL))
                    W_HVC_BG[...] = np.minimum(Wmax_RL, np.maximum(Wmin_RL, W_HVC_BG + dW_RL))
                

                # For tracking purposes
                R[n_syll, n_iter] = reward
                W_RL[nt] = W_HVC_BG[:n_bits,:].ravel()                                
                W_HL[nt] = W_HVC_RA.ravel()
                S_MO[nt] = MC.ravel()
                S_T[nt] = target
                S_RA[nt] = RA
                S_RA_HVC[nt] = RA_HVC
                S_RA_BG[nt] = RA_BG
                S_BG[nt] = BG
                S_dW1_sum[n_syll, n_iter] = day_dW1_sum[n_syll]
        

        ## Simulating nightly jumps within BG pathway
        if annealing == True:
            # Night shuffling of HVC-BG synapses
            p = day_dW1_sum[n_syll]/10                                                                      # p is proportional to daily sum of change in BG weights
            p = sigmoid(p, 10)

            # When potentiation is high during a day, we simulate a low disturbance in the night 
            potentiation_factor = np.zeros((HVC_size))
            potentiation_factor[:(n_bits*n_syllables)] = 1-p                                                # potentiation_factor is inversely proportional to p

            S_p[nt] = 1-p 

            night_noise[...] = np.random.uniform(-1, 1, (BG_size)) * BG_noise_std * 1000
            dW_night[...] = eta * (potentiation_factor.reshape(HVC_size,1)) * (night_noise) * BG_influence
            print(np.max(dW_night))
            W_HVC_BG[...] = (W_HVC_BG + dW_night + 1)%2-1                                                   # modulus so that the high noise doesn't make the weights to saturate


    if plotting == False: # change it to true
        # Save results for plotting
        np.savetxt('Results/'+resname+'_endRA.txt', S_RA[-1]>.5)

        test_k = 10                                                                                         # Chooses which particular RA neuron to observe

        # # Plot inputs to RA_exc
        # plt.close()

        # plt.plot(S_RA_BG[:,test_k], label='BG',lw=0.5, marker=',', alpha=.5)
        # # plt.plot(S_RA_inh[:,test_k], label='inh',lw=0.5, marker=',', alpha=.5)
        # plt.plot(S_RA_BG[:,test_k]+S_RA_HVC[:,test_k], label='BG+HVC',lw=0.5, marker=',', alpha=.5)
        # # plt.plot(S_RA_BG[:,test_k]+S_RA_HVC[:,test_k]-S_RA_exc_inh[:,test_k], label='BG+HVC-inh',lw=0.5, marker=',', alpha=.5)
        # plt.plot(S_RA[:,test_k], label='sig(input)',lw=0, marker=',', alpha=.5)
        # plt.plot(S_RA_HVC[:,test_k], label='HVC',lw=0.5, marker=',')

        # plt.legend()

        # plt.savefig('Results/' + resname + '_S_RA_inputs.png')
        # plt.close()
        

        # Plot sum of dW1

        # fig = plt.figure()
        # plt.plot(S_p, lw=0, marker='.', label='pf')
        # plt.plot(S_dW1_sum.flatten(), lw=0.2, marker=',', label='daily sum')
        # plt.plot(R.flatten(), lw=0.2, marker=',', alpha=.1, label='R')

        # plt.legend()

        # plt.savefig('Results/' + resname + '_S_dW1_sum.png')
        # plt.close()


        # # Plot RA_exc vs BG
        # fig = plt.figure()
        # plt.hist(S_RA.ravel()[::100], label='RA', alpha=.5);
        # plt.hist(S_BG.ravel()[::100], label='BG', alpha=.5);

        # plt.legend()

        # plt.savefig('Results/' + resname + '_S_RA.png')
        # plt.close()



        # Display overall results
        plt.figure(figsize=(12, 8))
        plt.rc("ytick", labelsize="small")
        plt.rc("xtick", labelsize="small")

        n_subplots = 6

        ax = plt.subplot(n_subplots, 1, 1)
        ax.set_title("Testing RL with normalised sigmoid")
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        # Plot all trial rewards
        T = np.arange(len(R.T.flatten()))
        ax.plot(T[::10], R.T.flatten()[::10], marker="o", markersize=1.5, linewidth=0, alpha=.25,
                color="none", markeredgecolor="none", markerfacecolor="black")
        ax.set_xlabel("Trial #")
        ax.set_ylim(0, 1)

        ax = plt.subplot(n_subplots, 1, 2)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        T = np.arange(n_total_trials)
        ax.plot(T, W_RL[:,test_k:5000:1000], color='black', alpha=.5, linewidth=0.5)
        ax.set_ylabel("HVC - BG weights")
        ax.set_ylim(Wmin_RL-0.1, Wmax_RL+0.1)

        ax = plt.subplot(n_subplots, 1, 3)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        T = np.arange(n_total_trials)
        ax.plot(T, S_BG[:,::200], color='black', alpha=.5, marker=',', linewidth=0, markersize=0.5)
        ax.set_ylabel("BG output")
        ax.set_ylim(-1.1, 1.1)

        ax = plt.subplot(n_subplots, 1, 4)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        T = np.arange(n_total_trials)
        ax.plot(T, W_HL[:,test_k:5000:1000], color='black', alpha=.5, linewidth=0.5)
        ax.set_ylabel("HVC - RA weights")
        ax.set_ylim(Wmin_HL-0.1, Wmax_HL+0.1) 

        ax = plt.subplot(n_subplots, 1, 5)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        T = np.arange(n_total_trials)
        ax.plot(T, S_MO, color='black', alpha=.5, marker=',', linewidth=0, markersize=0.5)
        ax.plot(T, S_T, color='red', marker=',', linewidth=0, markersize=0.5)
        ax.set_xlabel("Trials #")
        ax.set_ylabel("Output")
        ax.set_ylim(-1, 1)

        ax = plt.subplot(n_subplots, 1, 6)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        T = np.arange(n_total_trials)
        ax.plot(T, S_RA[:,test_k], color='black', alpha=.1, marker=',', linewidth=0, markersize=0.5)
        # ax.plot(T, S_RA_inh, color='red', alpha=.1, marker=',', linewidth=0, markersize=0.5)
        ax.axhline(y=0, color='red', alpha=.5, marker=',', linewidth=0.5)
        ax.axhline(y=0.5, color='black', alpha=.5, marker=',', linewidth=0.5)
        ax.set_ylabel("RA activity")
        ax.set_ylim(-1.1, 1.1)


        plt.tight_layout()
        plt.savefig('Results/' + resname + '_overall.png')
        plt.close()


        # Plot trajectory on 2d plane
        """ Plots the trajectory of the model output and cortical pathway over the simulation. """

        for n_syll in np.arange(n_syllables):
            syll = syllables[n_syll]
            contour = syllable_contours[syll]
            target = syllable_targets[syll]


            figure = plt.figure(figsize=(8,8))
            ax = plt.subplot(frameon=True)

            sk = n_syllables

            # Plot reward contour as circles around hills    
            contour_plot = ax.contourf(contour, 30, extent=[-1,1,-1,1], cmap="gray_r", alpha=.25)
            contour_plot = ax.contour(contour, 10, extent=[-1,1,-1,1], colors="black",  alpha=.5, linewidths=0.5)

            # Plot reward contour as a color plot
            im = ax.imshow(contour, vmin=0, vmax=np.max(contour), cmap='Purples', extent=[-1,1,-1,1], origin='lower')

            # Plot trajectory
            list_colors = cm.plasma(np.linspace(0, 1, 5))
            for day_index in np.arange(n_days+n_lesioned_days):
                ax.plot(S_MO[n_syll+day_index*n_daily_motifs:n_syll+(day_index+1)*n_daily_motifs:sk,0],S_MO[n_syll+day_index*n_daily_motifs:n_syll+(day_index+1)*n_daily_motifs:sk,1], color=list_colors[day_index%5], lw=0, marker='.', alpha=0.1, markersize=2)
            
            # # Annotations
            ax.scatter(S_MO[n_syll,0],S_MO[n_syll,1], color="sienna", marker="o",
                    linewidths=3, facecolors="sienna", zorder=10, label='Initial point')
            ax.scatter(S_MO[-n_syllables+n_syll,0],S_MO[-n_syllables+n_syll,1], s=100, color="orange", marker="x",
                    linewidths=3, facecolors="orange", zorder=10, label='Final point')

            # Display colorbar
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.5)
            cbar = figure.colorbar(im, cax=cax, ticks=[0, .5, 1])
            cbar.set_label('Performance metric (R)', rotation=270, fontsize=25, labelpad=25)
            cbar.ax.tick_params(labelsize=20)

            ax.set_xlabel(r'$P_{\beta}$', fontsize=30)
            ax.set_ylabel(r'$P_{\alpha}$', fontsize=30)
            ax.set_xticks(np.linspace(-1, 1, 3))
            ax.set_yticks(np.linspace(-1, 1, 3))
            ax.tick_params(labelsize=25)

            plt.tight_layout()

            plt.savefig('Results/' + resname + '_trajectory.png')

            plt.close()


    # Returns terminal performance (avg performance over last training day, and avg performance on lesion day
    return(np.mean(R[0,60000-1000:60000-1]), np.mean(R[0,-1000:-1]))


##### Running multiple simulations

In [10]:
#### Sample simulation of learning with and without annealing for a given rseed (with descriptive plots)

build_and_simulate(491263, True, True)
build_and_simulate(491263, False, True)

NameError: name 'gc' is not defined

In [ ]:
#### Multiple simulation of learning with and without annealing (with or without descriptive plots)

# Master seed to control the starting seeds generated for each simulation in the test batch
overall_seed = 0
np.random.seed(overall_seed)
print('Overall seed: ', overall_seed)

Overall seed:  0


In [ ]:
# Generating starting seeds for each simulation in the test batch

n_simulations = 10
seeds = np.random.randint(0, 1e7, n_simulations)

In [ ]:
# Recording terminal performance of each simulation

performance_annealing = np.zeros(n_simulations)
performance_no_annealing = np.zeros(n_simulations)

In [ ]:
### Testing robustness across different initial model configurations and sensorimotor landscapes

for i in tqdm(np.arange(n_simulations)):
    rseed = seeds[i]

    ## Testing a given setting in both annealing and no annealing cases
    # parameter ->(seed, annealing  - yes/no, plots - yes/no)
    perf_score1 = build_and_simulate(rseed, True, True)                    
    perf_score2 = build_and_simulate(rseed, False, True)

    ## Terminal performance (mean performance on the last day)
    # perf_score1[0] = avg performance on the last day before lesion of BG contribution
    # perf_score1[1] = avg performance on the day post lesion of BG contribution
    performance_annealing[i] = perf_score1[1]
    performance_no_annealing[i] = perf_score2[1]

np.save('Results/performance_annealing.npy', performance_annealing)
np.save('Results/performance_no_annealing.npy', performance_no_annealing)

  0%|          | 0/10 [00:00<?, ?it/s]

8325804_annealing
[ 0.13866744 -0.03879846]
4.735062063968962
4.457497751291981
4.57443464779038
3.305241780940749
3.909071333354904
3.4069178990346636
4.516265743849683
3.018597387601245
3.138127494559857
3.538354156032709
4.510167599829307
2.571437254861359
4.4589387984949935
4.2671291475149316
3.9945586037711758
3.8077596621046292
3.4017907661207665
3.4718310360963898
4.085619604435101
3.9443512775287353
4.022586631549136
4.350853279149407
3.2952946287935525
4.589057760269122
4.324247560252708
3.8123815458781376
4.119642847632705
4.577312239422607
4.629258940992512
4.071165717203823
4.574346315471708
4.65879385436242
4.588797562861973
4.069173307785134
4.51187000250781
4.218301342829765
4.627916051787283
4.639236919783418
4.660737500668392
4.050637243557865
4.676031766139143
4.634337830463532
4.547058387451077
4.536806918025076
3.832361040899289
4.315014121217622
4.104164234288224
4.514544397908663
4.491700371478405
4.2331297844842055
4.699834322161371
4.715540389596366
4.7391667763

 10%|█         | 1/10 [00:28<04:15, 28.42s/it]

1484405_annealing
[ 0.49806912 -0.31132766]
4.324878111966856
3.725479045024026
3.9387028477442954
3.766076066903968
4.154273974634873
4.394682722379847
3.751205921251962
3.5182722348710165
4.378458807495182
3.949841513986165
3.1315307688435463
4.206399907374396
3.9670102080490848
3.941892871554083
4.099948506255076
4.531679112601361
2.520362512790468
4.662425145296526
4.18026970769684
4.15762802749003
4.418640763589298
3.7484801672274273
3.3451968615751233
4.8142466627548375
3.043952906256059
3.9133086779793333
3.619603378586207
4.398633262386063
4.200063122869203
3.9366311108012475
4.169808946860593
4.18927107960839
4.379352358853628
4.366283551455504
3.9460562058390303
4.082739468685214
4.679855523391912
3.813781029654383
3.5644546638104173
4.374009956529532
4.240058372477997
4.353050949603913
4.731375057114863
4.312636336001611
4.464997491140895
4.5842834663073795
4.310220667742253
4.817989720169603
4.35409065864211
3.1898530598863357
4.297696169468718
4.043328106377721
4.255059004

 20%|██        | 2/10 [00:59<03:59, 29.89s/it]

...

 80%|████████  | 8/10 [04:05<01:02, 31.01s/it]

374564_annealing
[-0.00112911  0.52627081]
4.664447909753806
3.061904324165811
4.408994333278449
4.088580406244108
3.008288202069352
3.764061421833256
3.369170517753007
4.288058640576338
4.142967679404823
3.9582821458252586
3.3925200537619866
3.7547356351302588
4.608182539171466
3.8055988810777848
3.7736188555268244
4.087618012165982
4.5320589126059065
3.9696580093977913
4.0802692041111985
3.667840974299634
3.8976235973847904
4.043753852412625
3.877838596222974
4.2116964210131815
3.932310625924004
2.8651326696324415
4.302973817596235
3.962073461681759
3.8706338568934733
4.2848181027017525
4.37708620649069
4.293487686070354
3.9742564211766926
4.056725854842205
4.515262303915269
4.568217777266314
4.463109204957785
4.280825474210007
4.023315922837222
4.167806284881343
3.447579718326955
4.409269203313118
3.9594080387474744
4.758484690170988
4.455604256399133
4.682509575345694
4.679679406595051
4.491068742500783
3.998135227415253
4.62854745515601
4.74584642882871
4.176063864945506
4.5500387

 90%|█████████ | 9/10 [04:35<00:30, 30.91s/it]

2832983_annealing
[-0.53099852  0.20654741]
4.645680753198585
3.0359656570198417
3.026332565544727
3.8594511135332823
4.1081099530167435
4.852966110360101
3.1739712541708105
4.260960157687952
4.63269942521783
4.4901398452383
3.411373974895944
4.524551550203061
3.5413487765825966
3.581482804145001
3.8605604307412014
3.532574882645419
3.7699098777333893
3.749541917461334
4.081081035711901
3.5942417651560383
4.031817948334319
3.161238660923241
4.249163388866677
4.560083475597202
4.381437731403147
3.541929564537414
3.415164258382675
4.456005489672065
4.040660141793257
3.383702590910127
3.8056616826494367
3.3781859158007306
2.7504703391606444
4.012596217645437
4.323959841294457
3.8914853454126965
4.117946207082528
4.388538045406405
4.234366814035374
2.9585282484636695
2.925625568445262
3.7317825753701888
4.0131035061726
4.540016057545559
3.7429428517637438
4.149592139218008
3.4253558708759364
4.560951856695333
3.942634403538882
4.499954773489043
3.4901660599919704
3.9298923072249656
4.17522

100%|██████████| 10/10 [05:04<00:00, 30.49s/it]


In [ ]:
## Proportion of simulations that find the global optimum

# 0.6 = maximum yield across all local optima
# If a simulation has a terminal performance > 0.6 => the simulation must have landed on the global optimum region

overall_perf_annealing = np.count_nonzero(performance_annealing>.6)/n_simulations * 100
overall_perf_no_annealing = np.count_nonzero(performance_no_annealing>.6)/n_simulations * 100

plt.close()


In [ ]:
# Plot comparison of terminal performances with and without annealing

plt.plot(performance_annealing, linewidth=0, marker='.', color='g', label='Annealing')
plt.plot(performance_no_annealing, linewidth=0, marker='.', color='k', label='No annealing')

plt.legend()

plt.ylim(0,1)
plt.title('Score: ' + str(overall_perf_annealing) + '%,' + str(overall_perf_no_annealing) + '%')

plt.savefig('Results/Overall.png')

plt.close()